# Holdout Validation: 2024-25 Season

**True holdout test using the 2024-25 season**

Approach:
- ✅ Train on 2020-2024 seasons (27,794 games)
- ✅ Validate on 2024-25 season (~5,952 games)
- ✅ Compare predictions to actual scores
- ✅ Measure MAE, RMSE, and spread accuracy

This provides a realistic estimate of production performance.

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from src import config
from src.elo import EloRatingSystem
from src.features import FeatureEngine
from src.models import ImprovedSpreadModel
from src.utils import fetch_barttorvik_year
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("Libraries loaded!")

Libraries loaded!


## 1. Load Historical Games and Split by Season

In [2]:
# Load all historical games
games = pd.read_csv(config.HISTORICAL_GAMES_FILE, parse_dates=['date'])

print(f"Loaded {len(games)} total games")
print(f"Date range: {games['date'].min()} to {games['date'].max()}")
print(f"\nGames per season:")
print(games['season'].value_counts().sort_index())

# Split into train (2020-2024) and validation (2025)
train_games = games[games['season'] < 2025].copy()
val_games = games[games['season'] == 2025].copy()

print(f"\n{'='*60}")
print(f"Training set:   {len(train_games)} games (2020-2024)")
print(f"Validation set: {len(val_games)} games (2025)")
print(f"{'='*60}")

Loaded 33746 total games
Date range: 2019-11-05 00:00:00 to 2025-03-08 00:00:00

Games per season:
season
2020    5747
2021    4338
2022    5661
2023    6250
2024    5798
2025    5952
Name: count, dtype: int64

Training set:   27794 games (2020-2024)
Validation set: 5952 games (2025)


## 2. Process Elo Ratings Chronologically (All Games)

**Important**: We process Elo through ALL games to maintain temporal consistency, but only use pre-game ratings for prediction.

In [3]:
# Initialize Elo system
elo = EloRatingSystem(
    k_factor=config.ELO_CONFIG['k_factor'],
    hca=config.ELO_CONFIG['home_court_advantage'],
    carryover=config.ELO_CONFIG['season_carryover']
)
elo.load_conference_mappings(config.CONFERENCE_MAPPINGS)

# Process ALL games chronologically to build Elo history
print("Processing games chronologically to build Elo history...")
print("This may take a minute...\n")

elo_snapshots = elo.process_games(
    games,  # All games, including validation set
    date_col='date',
    home_col='home_team',
    away_col='away_team',
    home_score_col='home_score',
    away_score_col='away_score',
    neutral_col='neutral_site',
    season_col='season',
    save_snapshots=True
)

print(f"✓ Processed {len(elo_snapshots)} games")
print(f"✓ Tracked {len(elo.ratings)} team Elo ratings")

Processing games chronologically to build Elo history...
This may take a minute...

✓ Processed 33746 games
✓ Tracked 1213 team Elo ratings


## 3. Load Team Stats for Train/Val Seasons

In [4]:
# Fetch team stats for all years (2020-2025)
all_stats = []
for year in [2020, 2021, 2022, 2023, 2024, 2025]:
    print(f"Fetching {year}...")
    df = fetch_barttorvik_year(year)
    df['season'] = year
    all_stats.append(df[['team', 'adjoe', 'adjde', 'season']])

team_stats = pd.concat(all_stats, ignore_index=True)
team_stats.columns = ['team', 'adj_oe', 'adj_de', 'season']
team_stats['adj_em'] = team_stats['adj_oe'] - team_stats['adj_de']

print(f"\nLoaded efficiency stats for {len(team_stats)} team-seasons")

Fetching 2020...
Fetching 2021...
Fetching 2022...
Fetching 2023...
Fetching 2024...
Fetching 2025...

Loaded efficiency stats for 2147 team-seasons


## 4. Create Training Features (2020-2024 Only)

In [5]:
def create_features(elo_snaps, team_stats_df, feature_cols):
    """Helper to create features from elo snapshots and team stats"""
    data = elo_snaps.copy()
    data['season'] = data['date'].dt.year
    
    # Merge home team stats
    data = data.merge(
        team_stats_df,
        left_on=['home_team', 'season'],
        right_on=['team', 'season'],
        how='left',
        suffixes=('', '_home')
    )
    data = data.rename(columns={'adj_oe': 'home_adj_oe', 'adj_de': 'home_adj_de', 'adj_em': 'home_adj_em'})
    data = data.drop(columns=['team'], errors='ignore')
    
    # Merge away team stats
    data = data.merge(
        team_stats_df,
        left_on=['away_team', 'season'],
        right_on=['team', 'season'],
        how='left',
        suffixes=('', '_away')
    )
    data = data.rename(columns={'adj_oe': 'away_adj_oe', 'adj_de': 'away_adj_de', 'adj_em': 'away_adj_em'})
    data = data.drop(columns=['team'], errors='ignore')
    
    # Calculate derived features
    data['eff_diff'] = data['home_adj_em'] - data['away_adj_em']
    data['elo_diff'] = data['home_elo_before'] - data['away_elo_before']
    
    # Drop rows with missing data
    data = data.dropna(subset=['home_adj_oe', 'away_adj_oe'])
    
    return data

# Create training data
train_snaps = elo_snapshots[elo_snapshots['date'].dt.year < 2025]
train_data = create_features(train_snaps, team_stats, config.BASELINE_FEATURES)

X_train = train_data[config.BASELINE_FEATURES]
y_train = train_data['actual_margin']

print(f"✓ Training data: {len(X_train)} games")
print(f"  Features: {X_train.shape[1]}")
print(f"  Margin - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")

✓ Training data: 6802 games
  Features: 11
  Margin - Mean: -0.26, Std: 15.35


## 5. Create Validation Features (2025 Season)

In [6]:
# Create validation data
val_snaps = elo_snapshots[elo_snapshots['date'].dt.year == 2025]
val_data = create_features(val_snaps, team_stats, config.BASELINE_FEATURES)

X_val = val_data[config.BASELINE_FEATURES]
y_val = val_data['actual_margin']

print(f"✓ Validation data: {len(X_val)} games")
print(f"  Features: {X_val.shape[1]}")
print(f"  Margin - Mean: {y_val.mean():.2f}, Std: {y_val.std():.2f}")

✓ Validation data: 2048 games
  Features: 11
  Margin - Mean: -0.06, Std: 14.14


## 6. Train Model on 2020-2024 Data

In [7]:
# Train model
print("Training ImprovedSpreadModel on 2020-2024 data...\n")

model = ImprovedSpreadModel(
    ridge_alpha=config.MODEL_CONFIG['ridge_alpha'],
    lgbm_params={
        'n_estimators': config.MODEL_CONFIG['n_estimators'],
        'max_depth': config.MODEL_CONFIG['max_depth'],
        'learning_rate': config.MODEL_CONFIG['learning_rate'],
    },
    weights=(
        config.MODEL_CONFIG['ridge_weight'],
        config.MODEL_CONFIG['lgbm_weight']
    ),
    use_lgbm=True
)

model.fit(X_train, y_train)
print("✓ Model trained!\n")

# Training set performance
print("Training set performance:")
train_preds = model.predict(X_train)
train_components = model.predict_components(X_train)

for name, preds in train_components.items():
    mae = np.abs(preds - y_train).mean()
    rmse = np.sqrt(((preds - y_train) ** 2).mean())
    print(f"  {name:12} MAE={mae:.3f}, RMSE={rmse:.3f}")

Training ImprovedSpreadModel on 2020-2024 data...

✓ Model trained!

Training set performance:
  ridge        MAE=5.993, RMSE=7.770
  lgbm         MAE=4.182, RMSE=5.727
  ensemble     MAE=4.641, RMSE=6.234


## 7. Validate on 2024-25 Season (Holdout Set)

In [8]:
# Generate predictions on validation set
val_preds = model.predict(X_val)
val_components = model.predict_components(X_val)

print("\n" + "="*60)
print("HOLDOUT VALIDATION RESULTS (2024-25 Season)")
print("="*60)

# Calculate metrics for each component
for name, preds in val_components.items():
    mae = mean_absolute_error(y_val, preds)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    print(f"{name:12} MAE={mae:.3f}, RMSE={rmse:.3f}")

# Add actual vs predicted to validation data
val_results = val_data[['date', 'home_team', 'away_team', 'actual_margin']].copy()
val_results['predicted_margin'] = val_preds
val_results['error'] = val_results['actual_margin'] - val_results['predicted_margin']
val_results['abs_error'] = np.abs(val_results['error'])

# Overall statistics
print(f"\n{'='*60}")
print(f"Overall Holdout MAE:  {val_results['abs_error'].mean():.3f}")
print(f"Overall Holdout RMSE: {np.sqrt((val_results['error']**2).mean()):.3f}")
print(f"Median Absolute Error: {val_results['abs_error'].median():.3f}")
print(f"{'='*60}")


HOLDOUT VALIDATION RESULTS (2024-25 Season)
ridge        MAE=5.725, RMSE=7.377
lgbm         MAE=4.863, RMSE=6.728
ensemble     MAE=5.033, RMSE=6.813

Overall Holdout MAE:  5.033
Overall Holdout RMSE: 6.813
Median Absolute Error: 3.864


## 8. Detailed Error Analysis

In [9]:
# Error distribution
print("\nError Distribution:")
print(f"  Mean error (bias): {val_results['error'].mean():.3f}")
print(f"  Std of errors:     {val_results['error'].std():.3f}")
print(f"  Min error:         {val_results['error'].min():.3f}")
print(f"  Max error:         {val_results['error'].max():.3f}")

# Accuracy by error bucket
print("\nPrediction Accuracy Buckets:")
print(f"  Within 3 pts:  {(val_results['abs_error'] <= 3).mean()*100:.1f}%")
print(f"  Within 5 pts:  {(val_results['abs_error'] <= 5).mean()*100:.1f}%")
print(f"  Within 7 pts:  {(val_results['abs_error'] <= 7).mean()*100:.1f}%")
print(f"  Within 10 pts: {(val_results['abs_error'] <= 10).mean()*100:.1f}%")
print(f"  Within 15 pts: {(val_results['abs_error'] <= 15).mean()*100:.1f}%")

# Worst predictions
print("\nWorst 10 Predictions:")
worst = val_results.nlargest(10, 'abs_error')[['date', 'home_team', 'away_team', 'actual_margin', 'predicted_margin', 'abs_error']]
print(worst.to_string(index=False))


Error Distribution:
  Mean error (bias): -0.503
  Std of errors:     6.796
  Min error:         -34.754
  Max error:         40.358

Prediction Accuracy Buckets:
  Within 3 pts:  39.8%
  Within 5 pts:  62.0%
  Within 7 pts:  77.4%
  Within 10 pts: 88.1%
  Within 15 pts: 96.1%

Worst 10 Predictions:
      date    home_team     away_team  actual_margin  predicted_margin  abs_error
2025-01-05       Kansas           UCF           51.0         10.642086  40.357914
2025-01-30      IU Indy Robert Morris          -53.0        -18.245989  34.754011
2025-01-15      Bradley   Indiana St.           53.0         18.980620  34.019380
2025-02-22         Duke      Illinois           43.0         10.841334  32.158666
2025-02-06     Portland   Santa Clara          -47.0        -15.104409  31.895591
2025-02-19 San Jose St.      Utah St.          -48.0        -17.752173  30.247827
2025-02-22  Chattanooga   The Citadel            1.0         29.943032  28.943032
2025-02-19          VMI       Wofford      

## 9. Spread Accuracy (Against the Spread)

In sports betting, "covering the spread" means the actual margin beats the predicted spread.

In [10]:
# Calculate spread accuracy
# Home team "covers" if actual_margin > predicted_margin
val_results['home_covers'] = val_results['actual_margin'] > val_results['predicted_margin']
val_results['away_covers'] = val_results['actual_margin'] < val_results['predicted_margin']
val_results['push'] = np.abs(val_results['actual_margin'] - val_results['predicted_margin']) < 0.5

print("\nSpread Coverage Analysis:")
print(f"  Home covers: {val_results['home_covers'].sum()} ({val_results['home_covers'].mean()*100:.1f}%)")
print(f"  Away covers: {val_results['away_covers'].sum()} ({val_results['away_covers'].mean()*100:.1f}%)")
print(f"  Pushes:      {val_results['push'].sum()} ({val_results['push'].mean()*100:.1f}%)")
print(f"\n  Ideal spread accuracy: 50% each side")
print(f"  Our bias: {abs(val_results['home_covers'].mean() - 0.5)*100:.1f}% from ideal")


Spread Coverage Analysis:
  Home covers: 958 (46.8%)
  Away covers: 1090 (53.2%)
  Pushes:      126 (6.2%)

  Ideal spread accuracy: 50% each side
  Our bias: 3.2% from ideal


## 10. Best Predictions

In [11]:
print("\nBest 10 Predictions:")
best = val_results.nsmallest(10, 'abs_error')[['date', 'home_team', 'away_team', 'actual_margin', 'predicted_margin', 'abs_error']]
print(best.to_string(index=False))


Best 10 Predictions:
      date     home_team        away_team  actual_margin  predicted_margin  abs_error
2025-02-24   Alabama A&M      Florida A&M           11.0         11.015399   0.015399
2025-02-18 North Florida          Stetson            8.0          7.984166   0.015834
2025-02-13    Lindenwood     Morehead St.           13.0         13.016120   0.016120
2025-02-15  UC Riverside UC Santa Barbara           12.0         11.982698   0.017302
2025-02-05      Colorado             Utah          -13.0        -13.021629   0.021629
2025-01-24     Marquette        Villanova           13.0         12.960547   0.039453
2025-01-30     Stonehill           Wagner           12.0         11.960323   0.039677
2025-02-08       Indiana         Michigan           -3.0         -2.958709   0.041291
2025-01-25 Detroit Mercy       Wright St.          -17.0        -16.948886   0.051114
2025-02-27           LIU           Wagner           13.0         13.051949   0.051949


## 11. Save Validation Results

In [12]:
# Save detailed validation results
output_file = config.DATA_DIR.parent / 'outputs' / 'holdout_validation_2025.csv'
val_results.to_csv(output_file, index=False)
print(f"\n✓ Saved validation results to: {output_file}")

# Summary statistics
summary = {
    'training_games': len(X_train),
    'validation_games': len(X_val),
    'train_mae': mean_absolute_error(y_train, train_preds),
    'val_mae': mean_absolute_error(y_val, val_preds),
    'val_rmse': np.sqrt(mean_squared_error(y_val, val_preds)),
    'val_median_ae': val_results['abs_error'].median(),
    'within_3pts': (val_results['abs_error'] <= 3).mean(),
    'within_5pts': (val_results['abs_error'] <= 5).mean(),
    'within_7pts': (val_results['abs_error'] <= 7).mean(),
    'within_10pts': (val_results['abs_error'] <= 10).mean(),
}

print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
for key, val in summary.items():
    if 'within' in key:
        print(f"{key:20s}: {val*100:>6.1f}%")
    elif isinstance(val, int):
        print(f"{key:20s}: {val:>6d}")
    else:
        print(f"{key:20s}: {val:>6.3f}")
print("="*60)


✓ Saved validation results to: /Users/calebhan/Documents/Coding/Personal/triangle-sports-analytics-26/notebooks/../outputs/holdout_validation_2025.csv

FINAL SUMMARY
training_games      :   6802
validation_games    :   2048
train_mae           :  4.641
val_mae             :  5.033
val_rmse            :  6.813
val_median_ae       :  3.864
within_3pts         :   39.8%
within_5pts         :   62.0%
within_7pts         :   77.4%
within_10pts        :   88.1%


## 12. Generate 2025-26 Predictions

Now we'll use the model trained on 2020-2024 data to predict the 2025-26 season.

In [13]:
# Load 2026 prediction template and team stats
team_stats_2026 = pd.read_csv(config.PROCESSED_DATA_DIR / 'team_stats_2025_26.csv')
template = pd.read_csv(config.DATA_DIR.parent / config.SUBMISSION_TEMPLATE)
template = template.dropna(subset=['Home', 'Away'])

print(f"Teams for 2026: {len(team_stats_2026)}")
print(f"Games to predict: {len(template)}")
print(f"\nTeam stats columns: {list(team_stats_2026.columns)}")
team_stats_2026.head()

Teams for 2026: 21
Games to predict: 78

Team stats columns: ['team', 'off_efficiency', 'def_efficiency', 'ppg', 'opp_ppg', 'pace', 'power_rating', 'win_pct']


,team,off_efficiency,def_efficiency,ppg,opp_ppg,pace,power_rating,win_pct
0,Michigan,129.347488,91.066691,90.543241,63.746683,70.0,38.280797,0.885784
1,Virginia,126.385313,96.769961,88.469719,67.738973,70.0,29.615352,0.865642
2,Duke,124.200100,95.246451,86.940070,66.672516,70.0,28.953649,0.857254
3,Louisville,127.736968,99.549656,89.415878,69.684759,70.0,28.187312,0.719977
4,Clemson,118.186356,96.050643,82.730449,67.235450,70.0,22.135713,0.757256


In [14]:
# Create prediction features for 2026 games
team_dict = team_stats_2026.set_index('team').to_dict('index')

pred_features = []
valid_indices = []

for idx, row in template.iterrows():
    home = row['Home']
    away = row['Away']
    
    if home not in team_dict or away not in team_dict:
        print(f"Skipping {home} vs {away} - missing team stats")
        continue
    
    home_stats = team_dict[home]
    away_stats = team_dict[away]
    
    # Get efficiency stats
    home_oe = home_stats.get('off_efficiency', 100)
    home_de = home_stats.get('def_efficiency', 100)
    away_oe = away_stats.get('off_efficiency', 100)
    away_de = away_stats.get('def_efficiency', 100)
    
    # Build feature dictionary matching training features
    features = {
        'home_adj_oe': home_oe,
        'home_adj_de': home_de,
        'home_adj_em': home_oe - home_de,
        'away_adj_oe': away_oe,
        'away_adj_de': away_de,
        'away_adj_em': away_oe - away_de,
        'eff_diff': (home_oe - home_de) - (away_oe - away_de),
        'home_elo_before': elo.get_rating(home),
        'away_elo_before': elo.get_rating(away),
        'elo_diff': elo.get_rating(home) - elo.get_rating(away),
        'predicted_spread': elo.predict_spread(home, away),
    }
    
    pred_features.append(features)
    valid_indices.append(idx)

X_pred = pd.DataFrame(pred_features)
print(f"✓ Created features for {len(X_pred)} games")
print(f"  Feature columns: {list(X_pred.columns)}")
X_pred.head()

✓ Created features for 78 games
  Feature columns: ['home_adj_oe', 'home_adj_de', 'home_adj_em', 'away_adj_oe', 'away_adj_de', 'away_adj_em', 'eff_diff', 'home_elo_before', 'away_elo_before', 'elo_diff', 'predicted_spread']


,home_adj_oe,home_adj_de,home_adj_em,away_adj_oe,away_adj_de,away_adj_em,eff_diff,home_elo_before,away_elo_before,elo_diff,predicted_spread
0,126.385313,96.769961,29.615352,115.127068,102.420174,12.706895,16.908457,1805.093800,1772.246856,32.846944,5.173105
1,114.819162,102.750315,12.068847,127.736968,99.549656,28.187312,-16.118465,1942.734914,2131.343915,-188.609001,-2.736036
2,122.279937,100.661535,21.618402,117.038480,103.198697,13.839782,7.778620,1661.944737,1696.474229,-34.529492,2.766804
3,103.276594,102.083306,1.193289,117.728445,101.358278,16.370167,-15.176878,1548.381051,1500.000000,48.381051,5.727895
4,115.353556,108.769012,6.584544,125.606263,105.160202,20.446061,-13.861517,1500.000000,1877.537211,-377.537211,-9.483472


In [15]:
# Generate predictions using the trained model
predictions = model.predict(X_pred)
components = model.predict_components(X_pred)

# Add predictions to template
results = template.copy()
for i, idx in enumerate(valid_indices):
    results.loc[idx, 'pt_spread'] = predictions[i]
    results.loc[idx, 'ridge_pred'] = components['ridge'][i]
    results.loc[idx, 'lgbm_pred'] = components['lgbm'][i]
    results.loc[idx, 'elo_spread'] = X_pred.iloc[i]['predicted_spread']

print("✓ Predictions generated!")
print(f"\nFirst 15 predictions:")
results[['Date', 'Away', 'Home', 'pt_spread', 'ridge_pred', 'lgbm_pred', 'elo_spread']].head(15)

✓ Predictions generated!

First 15 predictions:


,Date,Away,Home,pt_spread,ridge_pred,lgbm_pred,elo_spread
0,2/7/2026,Syracuse,Virginia,14.656873,11.607885,15.963582,5.173105
1,2/7/2026,Louisville,Wake Forest,1.211571,-1.412022,2.335968,-2.736036
2,2/7/2026,Virginia Tech,NC State,8.170786,7.675504,8.383049,2.766804
3,2/7/2026,Miami,Boston College,8.832117,4.514719,10.682431,5.727895
4,2/7/2026,SMU,Pitt,-8.746583,-6.168425,-9.851507,-9.483472
5,2/7/2026,Florida State,Notre Dame,11.136124,13.379563,10.174650,12.548603
6,2/7/2026,Duke,North Carolina,-0.696340,-2.074428,-0.105730,-5.839441
7,2/7/2026,Clemson,California,-9.397259,-8.198705,-9.910925,-13.508222
8,2/7/2026,Georgia Tech,Stanford,10.831013,10.195143,11.103528,6.379343
9,2/9/2026,NC State,Louisville,24.287374,20.720617,25.815984,20.764256


In [16]:
# Prepare submission file
submission = results[['Date', 'Away', 'Home', 'pt_spread']].copy()
submission = submission.dropna(subset=['pt_spread'])

# Add team info from config
submission['team_name'] = ''
submission['team_member'] = ''
submission['team_email'] = ''

team_members = config.TEAM_INFO['members']
submission.loc[submission.index[0], 'team_name'] = config.TEAM_INFO['team_name']
for i, member in enumerate(team_members):
    if i < len(submission):
        submission.loc[submission.index[i], 'team_member'] = member['name']
        submission.loc[submission.index[i], 'team_email'] = member['email']

# Save to a different filename to distinguish from the full-data model
output_file = config.DATA_DIR.parent / 'data' / 'predictions' / 'tsa_pt_spread_CMMT_2026_holdout.csv'
submission.to_csv(output_file, index=False)

print(f"✓ Saved 2025-26 predictions to: {output_file}")
print(f"✓ Total predictions: {len(submission)}")

✓ Saved 2025-26 predictions to: /Users/calebhan/Documents/Coding/Personal/triangle-sports-analytics-26/notebooks/../data/predictions/tsa_pt_spread_CMMT_2026_holdout.csv
✓ Total predictions: 78


In [17]:
# Final summary
print("\n" + "="*70)
print("COMPLETE HOLDOUT VALIDATION AND 2025-26 PREDICTION SUMMARY")
print("="*70)
print(f"\n📊 Training Data (2020-2024):")
print(f"   Games trained on:     {len(X_train):,}")
print(f"   Training MAE:         {mean_absolute_error(y_train, train_preds):.3f}")

print(f"\n🧪 Holdout Validation (2024-25):")
print(f"   Games validated:      {len(X_val):,}")
print(f"   Validation MAE:       {mean_absolute_error(y_val, val_preds):.3f}")
print(f"   Validation RMSE:      {np.sqrt(mean_squared_error(y_val, val_preds)):.3f}")
print(f"   Median Absolute Error: {val_results['abs_error'].median():.3f}")
print(f"\n   Accuracy:")
print(f"     Within 3 pts:  {(val_results['abs_error'] <= 3).mean()*100:.1f}%")
print(f"     Within 5 pts:  {(val_results['abs_error'] <= 5).mean()*100:.1f}%")
print(f"     Within 7 pts:  {(val_results['abs_error'] <= 7).mean()*100:.1f}%")
print(f"     Within 10 pts: {(val_results['abs_error'] <= 10).mean()*100:.1f}%")

print(f"\n🔮 2025-26 Predictions:")
print(f"   Games predicted:      {len(submission)}")
print(f"   Output file:          {output_file.name}")

print(f"\n✅ Model Performance:")
print(f"   Generalization gap:   {mean_absolute_error(y_val, val_preds) - mean_absolute_error(y_train, train_preds):.3f} points")
print(f"   Spread calibration:   {abs(val_results['home_covers'].mean() - 0.5)*100:.1f}% bias from ideal")

print("="*70)
print("\n🎯 Key Insight: Model trained on 2020-2024 achieves ~5.0 MAE on")
print("   unseen 2024-25 data, demonstrating strong generalization.")
print("="*70)


COMPLETE HOLDOUT VALIDATION AND 2025-26 PREDICTION SUMMARY

📊 Training Data (2020-2024):
   Games trained on:     6,802
   Training MAE:         4.641

🧪 Holdout Validation (2024-25):
   Games validated:      2,048
   Validation MAE:       5.033
   Validation RMSE:      6.813
   Median Absolute Error: 3.864

   Accuracy:
     Within 3 pts:  39.8%
     Within 5 pts:  62.0%
     Within 7 pts:  77.4%
     Within 10 pts: 88.1%

🔮 2025-26 Predictions:
   Games predicted:      78
   Output file:          tsa_pt_spread_CMMT_2026_holdout.csv

✅ Model Performance:
   Generalization gap:   0.392 points
   Spread calibration:   3.2% bias from ideal

🎯 Key Insight: Model trained on 2020-2024 achieves ~5.0 MAE on
   unseen 2024-25 data, demonstrating strong generalization.
